# Introduction

In this notebook that scores 0.650, we to each breath in the test file assign the average of pressures from several breaths in the train file that
1. have the same R and C values as the given test breath
2. have the most similar u_in as measured by chi squared value

We load the nearest neighbor breath_id precomputed in [this notebook](https://www.kaggle.com/motloch/ventilator-pressure-find-nn-u-in-same-r-c).

We build on the [exploratory analysis](https://www.kaggle.com/motloch/ventilator-pressure-train-data-exploration) we did earlier.

Version 4: Average several nearest neighbors

# Import libraries

In [ ]:
import numpy as np
import pandas as pd

# Load data sets, sample submission and nearest neighbors for each test breath

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
nn_breath_id = np.loadtxt('../input/ventilator-pressure-find-nn-u-in-same-r-c/nn.txt').astype(int)

# Number of data points per breath, number of breaths in test and train data

In [ ]:
BREATH_LENGTH = 80
num_train_breaths = len(train) // BREATH_LENGTH
num_test_breaths  = len(test)  // BREATH_LENGTH

# Dictionary that converts train breath id to position of the first element in the array (trick to speed up)

In [ ]:
starts = {}
for idx in range(num_train_breaths):
    starts[train['breath_id'][idx*BREATH_LENGTH]] = idx*BREATH_LENGTH

# For each test breath, average pressure from the train breaths with the same R/C and closest u_in

In [ ]:
USE_NN = 3 #How many NN to average over

for idx in range(num_test_breaths):
    
    p = np.zeros(BREATH_LENGTH)
    
    for j in range(USE_NN):
        train_start = starts[nn_breath_id[idx][j]]
        p += train['pressure'][train_start:train_start + BREATH_LENGTH].values / USE_NN
        
    sub['pressure'][BREATH_LENGTH*idx:BREATH_LENGTH*(idx+1)] = p

# Check and save

In [ ]:
sub

In [ ]:
sub.to_csv('final_submission.csv', index=False)